In [419]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.signal import savgol_filter
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import zscore
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Preprocessing the dataset

In [420]:
train_df = pd.read_csv(r'E:\NB\Machine Learning\UOM - DataCrunch\RoboZen404_DataCrunch\dataset\train.csv') #replace with your file name.
test_df = pd.read_csv(r'E:\NB\Machine Learning\UOM - DataCrunch\RoboZen404_DataCrunch\dataset\test.csv')

# Load your train and test datasets
train_df['is_test'] = 0  # Mark training data
test_df['is_test'] = 1   # Mark test data

Check for missing values

In [421]:
# List of missing columns in test data
missing_cols = ['latitude', 'longitude', 'Avg_Temperature', 'Avg_Feels_Like_Temperature', 
                'Temperature_Range', 'Feels_Like_Temperature_Range', 'Radiation', 
                'Rain_Amount', 'Rain_Duration', 'Wind_Speed', 'Wind_Direction', 'Evapotranspiration']

# Add them with NaN values in test data
for col in missing_cols:
    test_df[col] = np.nan

print(train_df.shape)
print(train_df.head())

print(test_df.shape)
print(test_df.head())

(84960, 18)
   ID  Year  Month  Day   kingdom   latitude  longitude  Avg_Temperature  \
0   1     1      4    1   Arcadia  24.280002 -37.229980            25.50   
1   2     1      4    1  Atlantis  22.979999 -37.329990           299.65   
2   3     1      4    1    Avalon  22.880000 -37.130006            26.30   
3   4     1      4    1   Camelot  24.180003 -36.929994            24.00   
4   5     1      4    1     Dorne  25.780002 -37.530000            28.00   

   Avg_Feels_Like_Temperature  Temperature_Range  \
0                       30.50                8.5   
1                      305.15                5.9   
2                       31.50                5.2   
3                       28.40                8.2   
4                       32.80                5.7   

   Feels_Like_Temperature_Range  Radiation  Rain_Amount  Rain_Duration  \
0                          10.3      22.52        58.89             16   
1                           8.2      22.73        11.83             12

In [422]:
print(train_df.isnull().sum())

ID                              0
Year                            0
Month                           0
Day                             0
kingdom                         0
latitude                        0
longitude                       0
Avg_Temperature                 0
Avg_Feels_Like_Temperature      0
Temperature_Range               0
Feels_Like_Temperature_Range    0
Radiation                       0
Rain_Amount                     0
Rain_Duration                   0
Wind_Speed                      0
Wind_Direction                  0
Evapotranspiration              0
is_test                         0
dtype: int64


There are no any missing values in the train dataset. So we can concatinate

In [423]:
# Concatenate train and test data
full_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)

Scalling the dataset

In [424]:
# Separate out date-related columns (Year, Month, Day) from numerical columns
date_cols = ['Year', 'Month', 'Day', 'is_test']  # Adjust based on your dataset

# Get only numerical columns that are not date-related
numerical_cols = [col for col in full_df.select_dtypes(include=np.number).columns if col not in date_cols]

# Apply StandardScaler or MinMaxScaler only to the numerical columns
scaler = StandardScaler()
full_df[numerical_cols] = scaler.fit_transform(full_df[numerical_cols])

# Alternatively, use MinMaxScaler if preferred
# scaler = MinMaxScaler()
# full_df[numerical_cols] = scaler.fit_transform(full_df[numerical_cols])

# Check the results to confirm scaling
print(full_df[numerical_cols].describe())


                 ID      latitude     longitude  Avg_Temperature  \
count  8.949000e+04  8.496000e+04  8.496000e+04     8.496000e+04   
mean  -6.097853e-17  1.784992e-15 -1.646687e-14    -9.697202e-17   
std    1.000006e+00  1.000006e+00  1.000006e+00     1.000006e+00   
min   -1.732031e+00 -1.406599e+00 -9.477272e-01    -8.754289e-01   
25%   -8.660157e-01 -4.048639e-01 -7.432270e-01    -8.178156e-01   
50%    0.000000e+00 -2.796493e-01 -5.386612e-01    -8.043475e-01   
75%    8.660157e-01  3.464329e-01  2.795403e-01     1.225213e+00   
max    1.732031e+00  3.226416e+00  3.143342e+00     1.257387e+00   

       Avg_Feels_Like_Temperature  Temperature_Range  \
count                8.496000e+04       8.496000e+04   
mean                -8.346535e-17       1.552221e-16   
std                  1.000006e+00       1.000006e+00   
min                 -9.036781e-01      -2.449927e+00   
25%                 -8.170698e-01      -7.813450e-01   
50%                 -8.006441e-01      -1.240248e-0

In [425]:
window_size = 3  # Adjust based on need

smooth_features = ['Avg_Temperature', 'Avg_Feels_Like_Temperature', 
                   'Temperature_Range', 'Feels_Like_Temperature_Range',
                   'Radiation', 'Evapotranspiration']

for feature in smooth_features:
    full_df[feature] = full_df[feature].rolling(window=window_size, min_periods=1).mean()


In [426]:
alpha = 0.3  # Smoothing factor (0 < alpha < 1)

for feature in smooth_features:
    full_df[feature] = full_df[feature].ewm(alpha=alpha, adjust=False).mean()

Outlier removal

In [427]:
# Check first few rows of full_df
print(full_df.head())

         ID  Year  Month  Day   kingdom  latitude  longitude  Avg_Temperature  \
0 -1.732031     1      4    1   Arcadia  0.346433   0.075040        -0.823801   
1 -1.731993     1      4    1  Atlantis -1.281385  -0.129534        -0.516113   
2 -1.731954     1      4    1    Avalon -1.406599   0.279540        -0.402695   
3 -1.731915     1      4    1   Camelot  0.221218   0.688672        -0.324425   
4 -1.731877     1      4    1     Dorne  2.224680  -0.538661        -0.472891   

   Avg_Feels_Like_Temperature  Temperature_Range  \
0                   -0.815577           1.595120   
1                   -0.507987           1.397924   
2                   -0.394457           1.158761   
3                   -0.316554           0.976178   
4                   -0.465365           0.838257   

   Feels_Like_Temperature_Range  Radiation  Rain_Amount  Rain_Duration  \
0                      1.660623   0.529606     3.796523       0.982415   
1                      1.527817   0.537254     0.304

In [428]:
# Step 1: Split full_df into train and test
train_df = full_df[full_df['is_test'] == 0].drop(columns=['is_test'])
test_df = full_df[full_df['is_test'] == 1].drop(columns=['is_test', 'Avg_Temperature', 'Radiation', 'Rain_Amount', 'Wind_Speed', 'Wind_Direction'])


scaled_features = ['Avg_Temperature', 'Avg_Feels_Like_Temperature', 'Temperature_Range',
                   'Feels_Like_Temperature_Range', 'Radiation', 'Rain_Amount', 'Rain_Duration',
                   'Wind_Speed', 'Evapotranspiration']


scaled_features = [feature for feature in scaled_features if feature in full_df.columns]



In [429]:
print(train_df.shape)
print(train_df.head())

(84960, 17)
         ID  Year  Month  Day   kingdom  latitude  longitude  Avg_Temperature  \
0 -1.732031     1      4    1   Arcadia  0.346433   0.075040        -0.823801   
1 -1.731993     1      4    1  Atlantis -1.281385  -0.129534        -0.516113   
2 -1.731954     1      4    1    Avalon -1.406599   0.279540        -0.402695   
3 -1.731915     1      4    1   Camelot  0.221218   0.688672        -0.324425   
4 -1.731877     1      4    1     Dorne  2.224680  -0.538661        -0.472891   

   Avg_Feels_Like_Temperature  Temperature_Range  \
0                   -0.815577           1.595120   
1                   -0.507987           1.397924   
2                   -0.394457           1.158761   
3                   -0.316554           0.976178   
4                   -0.465365           0.838257   

   Feels_Like_Temperature_Range  Radiation  Rain_Amount  Rain_Duration  \
0                      1.660623   0.529606     3.796523       0.982415   
1                      1.527817   0.5372

In [430]:
z_threshold = 3  # Common threshold (|Z-score| > 3 is considered an outlier)
full_df = full_df[(np.abs(zscore(full_df[scaled_features])) < z_threshold).all(axis=1)]

print("Dataset shape after outlier removal:", full_df.shape)

Dataset shape after outlier removal: (0, 18)


In [431]:
# Step 2: Concatenate the cleaned train_df and the test_df back into full_df
# We add the 'is_test' column again for clarity, if you need it later
train_df['is_test'] = 0  # Mark the train data
test_df['is_test'] = 1   # Mark the test data

# Concatenate train and test data back into full_df
full_df = pd.concat([train_df, test_df], ignore_index=True)

# Step 3: Check the shape and preview of the concatenated data
print("Full dataset shape after concatenation:", full_df.shape)
print(full_df.head())

Full dataset shape after concatenation: (89490, 18)
         ID  Year  Month  Day   kingdom  latitude  longitude  Avg_Temperature  \
0 -1.732031     1      4    1   Arcadia  0.346433   0.075040        -0.823801   
1 -1.731993     1      4    1  Atlantis -1.281385  -0.129534        -0.516113   
2 -1.731954     1      4    1    Avalon -1.406599   0.279540        -0.402695   
3 -1.731915     1      4    1   Camelot  0.221218   0.688672        -0.324425   
4 -1.731877     1      4    1     Dorne  2.224680  -0.538661        -0.472891   

   Avg_Feels_Like_Temperature  Temperature_Range  \
0                   -0.815577           1.595120   
1                   -0.507987           1.397924   
2                   -0.394457           1.158761   
3                   -0.316554           0.976178   
4                   -0.465365           0.838257   

   Feels_Like_Temperature_Range  Radiation  Rain_Amount  Rain_Duration  \
0                      1.660623   0.529606     3.796523       0.982415   


# Feature Engineering

Lag Features: We created lag features to capture temporal dependencies without relying on actual time. These included shifts of key variables like temperature, wind speed, and rain amount (e.g., 1-day lag).

Rolling Features: We introduced rolling statistics, such as moving averages and rolling sums, to identify trends over a specified window (e.g., 7-day moving averages for temperature and rain).

Seasonal Features: To account for cycles, we grouped the data into hypothetical seasons based on the "Year" variable, using a modulo operation to classify records into 4 seasons.

Aggregated Features by Kingdom: We aggregated key variables like temperature and wind speed by "kingdom" to capture regional patterns and trends.

External Variables: We considered adding external features like radiation levels or indicators based on thresholds (e.g., high radiation) to help the model capture important weather patterns.

In [432]:
# Creating a lag of 1 day for 'Avg_Temperature'
full_df['Avg_Temperature_Lag_1'] = full_df['Avg_Temperature'].shift(1)
full_df['Wind_Speed_Lag_1'] = full_df['Wind_Speed'].shift(1)


In [433]:
full_df['Avg_Temperature_Lag_2'] = full_df['Avg_Temperature'].shift(2)


In [434]:
# Moving average over 7 days for 'Avg_Temperature'
full_df['Avg_Temperature_MA_7'] = full_df['Avg_Temperature'].rolling(window=7).mean()
full_df['Rain_Amount_MA_7'] = full_df['Rain_Amount'].rolling(window=7).mean()


In [435]:
# Rolling sum over 7 days
full_df['Rain_Amount_Rolling_Sum_7'] = full_df['Rain_Amount'].rolling(window=7).sum()

# Rolling standard deviation
full_df['Wind_Speed_Rolling_Std_7'] = full_df['Wind_Speed'].rolling(window=7).std()


Seasonal Features (Non-time-based)

In [436]:
full_df['Season'] = (full_df['Year'] % 4)  # Group into 4 seasons based on the hypothetical year


Aggregated Features by Kingdom

In [437]:
full_df['Avg_Temperature_Kingdom_Mean'] = full_df.groupby('kingdom')['Avg_Temperature'].transform('mean')
full_df['Wind_Speed_Kingdom_Mean'] = full_df.groupby('kingdom')['Wind_Speed'].transform('mean')


External Variables

In [438]:
full_df['High_Radiation'] = (full_df['Radiation'] > full_df['Radiation'].median()).astype(int)


In [439]:
from statsmodels.tsa.stattools import adfuller

#result = adfuller(full_df['Avg_Temperature'])
#print(f'ADF Statistic: {result[0]}')
#print(f'p-value: {result[1]}')

Above values have tested ans since the ADF statistic is -31.50 and the p-value is 0.0, this means that the data is stationary. No need to do any transformations

Split the data into train and test

In [ ]:
### --- SPLIT BACK INTO TRAIN AND TEST --- ###
train_df = full_df[full_df['is_test'] == 0].drop(columns=['is_test'])
test_df = full_df[full_df['is_test'] == 1].drop(columns=['is_test', 'Avg_Temperature', 'Radiation', 'Rain_Amount', 'Wind_Speed', 'Wind_Direction'])

In [441]:
print(train_df.shape)
print(train_df.head())

# print(test_df.shape)
# print(test_df.head())

(84960, 28)
         ID  Year  Month  Day   kingdom  latitude  longitude  Avg_Temperature  \
0 -1.732031     1      4    1   Arcadia  0.346433   0.075040        -0.823801   
1 -1.731993     1      4    1  Atlantis -1.281385  -0.129534        -0.516113   
2 -1.731954     1      4    1    Avalon -1.406599   0.279540        -0.402695   
3 -1.731915     1      4    1   Camelot  0.221218   0.688672        -0.324425   
4 -1.731877     1      4    1     Dorne  2.224680  -0.538661        -0.472891   

   Avg_Feels_Like_Temperature  Temperature_Range  ...  Wind_Speed_Lag_1  \
0                   -0.815577           1.595120  ...               NaN   
1                   -0.507987           1.397924  ...         -1.133990   
2                   -0.394457           1.158761  ...          0.027539   
3                   -0.316554           0.976178  ...          0.027539   
4                   -0.465365           0.838257  ...         -1.488902   

   Avg_Temperature_Lag_2  Avg_Temperature_MA_7  Ra

# Test the model